# Search through gnps libraries to select dataset

In [2]:
import json
import os

In [3]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\SeSiMe\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\SeSiMe\\data\\"
PATH_SESIME = ROOT + "\\SeSiMe\\"

PATH_NPLINKER = ROOT + "\\nplinker\\prototype\\"

In [4]:
json_file = PATH_MS_DATA + 'gnpslibraryjson_filtered.json'

with open(json_file, 'rb') as f:
    data = json.load(f)

In [5]:
len(data)

77253

In [6]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

In [15]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [16]:
keys[:10]

['CCMSLIB00000001548',
 'CCMSLIB00000001552',
 'CCMSLIB00000001577',
 'CCMSLIB00000001660',
 'CCMSLIB00000001666',
 'CCMSLIB00000001674',
 'CCMSLIB00000001685',
 'CCMSLIB00000006838',
 'CCMSLIB00000006841',
 'CCMSLIB00000006864']

In [17]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Functions

In [18]:
def check_spectrum_lines(spectrum):
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
            
        if l.startswith('CHARGE') and l.strip().endswith('-1+'):
            return False
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename):
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            if line.startswith('END ION'):
                # Check if we have structural info
                if check_spectrum_lines(spectrum):
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
    return matched_spectra

In [19]:
path = PATH_MS_DATA + 'speclibs\\'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f))
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 0
CASMI.mgf 522
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 522
GNPS-COLLECTIONS-MISC.mgf 547
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 623
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1276
GNPS-EMBL-MCF.mgf 1764
GNPS-FAULKNERLEGACY.mgf 1765
GNPS-LIBRARY.mgf 3904
GNPS-NIH-CLINICALCOLLECTION1.mgf 4281
GNPS-NIH-CLINICALCOLLECTION2.mgf 4289
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 5556
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 5556
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 5556
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 7016
GNPS-NIST14-MATCHES.mgf 7085
GNPS-PRESTWICKPHYTOCHEM.mgf 7228
GNPS-SELLECKCHEM-FDA-PART1.mgf 7275
GNPS-SELLECKCHEM-FDA-PART2.mgf 7931
HMDB.mgf 10166
LDB_NEGATIVE.mgf 10166
LDB_POSITIVE.mgf 10166
MASSBANK.mgf 21692
MASSBANKEU.mgf 22996
MIADB.mgf 22996
MMV_NEGATIVE.mgf 22996
MMV_POSITIVE.mgf 22996
MONA.mgf 72237
PNNL-LIPIDS-NEGATIVE.mgf 72237
PNNL-LIPIDS-POSITIVE.mgf 72237
RESPECT.mgf 77102
SUMNER.mgf 77102


In [43]:
collected_spectra[0]

['BEGIN IONS\n',
 'TITLE=Theophyllin\n',
 'LIBRARYQUALITY=1\n',
 'SMILES=CN1C2=C(NC=N2)C(=O)N(C)C1=O\n',
 'INCHIKEY=ZFXYFBGIUFBOJW-UHFFFAOYSA-N\n',
 'INCHI="InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)"\n',
 'PEPMASS=179.057\n',
 'CHARGE=1\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=LC-ESI-Orbitrap\n',
 'FILENAME=Training_001.mgf\n',
 'SEQ=*..*\n',
 'IONMODE= Negative\n',
 'ORGANISM=CASMI\n',
 'NAME=Theophyllin M-H\n',
 'PI=CASMI\n',
 'DATACOLLECTOR=CASMI2016\n',
 'PUBMED=33919\n',
 'SUBMITUSER=lfnothias\n',
 'TAGS=\n',
 'SPECTRUMID=CCMSLIB00000579358\n',
 'SCANS=1\n',
 '164.033997\t32294.5\n',
 '179.057495\t129907.101562\n',
 'END IONS\n']

In [76]:
import sys

file_mgf = PATH_MS_DATA + 'all_gnps_with_smiles_190915.mgf'
with open(file_mgf, 'w') as f:
    for c in collected_spectra[:]:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        #f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=(1R,2S,5S,8R)-7,7-dimethyl-6-methylidenetricyclo[6.2.1.0�,?]undecane-2-carboxylic acid

TITLE=(1R,2S,5S,8R)-7,7-dimethyl-6-methylidenetricyclo[6.2.1.0�,?]undecane-2-carboxylic acid

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,2S,3R,5R,6S,8S)-6-hydroxy-8-methyl-3-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate


TITLE=(1R,2E,8S,10R,11S)-10,11-dihydroxy-6-(methoxymethyl)-1,10-dimethyl-5-oxo-4,14-dioxatricyclo[9.2.1.0�,?]tetradeca-2,6-dien-8-yl acetate

TITLE=(1S,2S,5R,7R,10S,11S,14R,16R,19S,20S,23R,25R)-5,14-diethyl-2,11,20,23-tetramethyl-4,13,22,28,29,30-hexaoxatetracyclo[23.2.1.1?,�?.1�?,�?]triacontane-3,12,21-trione

TITLE=(1S,2S,5R,7R,10S,11S,14R,16R,19S,20S,23R,25R)-5,14-diethyl-2,11,20,23-tetramethyl-4,13,22,28,29,30-hexaoxatetracyclo[23.2.1.1?,�?.1�?,�?]triacontane-3,12,21-trione

TITLE=15,16-dimethoxytricyclo[12.3.1.1�,?]nonadeca-1(18),2,4,6(19),14,16-hexaene-3,9,17-triol

TITLE=8-hydroxy-2,7,7,11,15-pentamethyl-5,12,16-trioxapentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadec-13(18)-ene-3,17-dione

TITLE=8-hydroxy-2,7,7,11,15-pentamethyl-5,12,16-trioxapentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadec-13(18)-ene-3,17-dione

TITLE=8-hydroxy-2,7,7,11,15-pentamethyl-5,12,16-trioxapentacyclo[9.8.0.0�,?.0?,?.0��,�?]nonadec-13(18)-ene-3,17-dione

TITLE=8-hydroxy-2,7,7,11,15-pentamethyl-5,12,16-trioxapentacyclo

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4

...

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]doc


TITLE=(1S,6R,10S,13S,14R,15R)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-13-[(2-methylbutanoyl)oxy]-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-14-yl (2E)-2-methylbut-2-enoate

TITLE=(1S,6R,10S,13S,14R,15R)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-13-[(2-methylbutanoyl)oxy]-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-14-yl (2E)-2-methylbut-2-enoate

TITLE=(1S,6R,10S,13S,14R,15R)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-13-[(2-methylbutanoyl)oxy]-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-14-yl (2E)-2-methylbut-2-enoate

TITLE=(1S,6R,10S,13S,14R,15R)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-13-[(2-methylbutanoyl)oxy]-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-14-yl (2E)-2-methylbut-2-enoate

TITLE=(1S,6R,10S,13S,14R,15R)-1,6-dihydroxy-8-(hydroxymethyl)-4,12,12,15-tetramethyl-13-[(2-methylbutanoyl)oxy]-5-oxotetracyclo[8.5.0.0�,?.0��,��]pentadeca-3,8-dien-14-yl (2E)-2-methylbut-2-enoate

TITLE=(1S

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione



TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)ethyl]-11,20,21,25-tetrahydroxy-15-[(1R)-1-hydroxyethyl]-26-methyl-2,5,8,14,17,23-hexaoxo-1,4,7,13,16,22-hexaazatricyclo[22.3.0.0?,��]heptacosan-18-yl]-10,12-dimethyltetradecanamide

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)ethyl]-11,20,21,25-tetrahydroxy-15-[(1R)-1-hydroxyethyl]-26-methyl-2,5,8,14,17,23-hexaoxo-1,4,7,13,16,22-hexaazatricyclo[22.3.0.0?,��]heptacosan-18-yl]-10,12-dimethyltetradecanamide

TITLE=N-[(3S,6S,9S,11R,15S,18S,20R,21R,24S,25S,26S)-3-[(1R)-2-carbamoyl-1-hydroxyethyl]-6-[(1S,2S)-1,2-dihydroxy-2-(4-hydroxyphenyl)ethyl]-11,20,21,25-tetrahydroxy-15-[(1R)-1-hydroxyet

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethyl

In [74]:
sys.version_info

sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)